In [8]:
import pandas as pd

# Read File and create a new dataframe called 'df'
# df = pd.read_csv('dataset_1.csv')
df = pd.read_csv('full_tagged_dataset_2%.csv')

# Removing useless columns
# df = df.drop(columns = ['Unnamed: 0', 'link', 'source'])

df

,TITLE,NER,difficulty,time,cost,method,INGREDIENTS,DIRECTIONS,total_time,PREPARATION_TIME,NER_clean,VEGAN,NON_VEGAN_INGREDIENTS,VEGETARIAN,NON_VEGETARIAN_INGREDIENTS,total_cost,price_tag,categories,vegan,vegetarian
0,Western Sizzlin Bread Pudding,"['eggs', 'milk', 'sugar', 'vanilla', 'gallon w...",Hard,Slow,Cheap,Bake,"[""16 eggs, beaten"", ""6 -8 evaporated milk, 360...","[""Distribute rolls and cinnamon roll in (4) 2\...",150.0,Very slow (90+ mins),"['eggs', 'milk', 'sugar', 'vanilla', 'gallon w...",False,"['eggs', 'milk']",True,[],49.2,medium,"['seafood', 'dairy', 'sweetener', 'spice/herb'...",False,False
1,Creamy Tuna And Bacon Salad (Reduced Fat),"['tuna', 'mayonnaise', 'low-fat sour cream', '...",Hard,Average,Cheap,Other,"[""1 (6 ounce) can tuna, drained and flaked"", ""...","[""Put everything in a bowl and mix together un...",15.0,Fast (10-20 mins),"['tuna', 'mayonnaise', 'low-fat sour cream', '...",False,"['tuna', 'low-fat sour cream', 'bacon bits']",False,"['tuna', 'bacon bits']",52.5,expensive,"['seafood', 'condiment', None, 'meat', 'spice/...",False,False
2,Spinach And Mushroom Lasagna,"['olive oil', 'onion', 'garlic', 'salt', 'grou...",Hard,Quick,Cheap,Bake,"[""1 tablespoon olive oil"", ""1 medium onion , c...","[""Heat oven to 375 degrees."", """", ""In medium s...",70.0,Slow (40-90 mins),"['olive oil', 'onion', 'garlic', 'salt', 'grou...",False,"['ricotta cheese', 'Parmesan cheese', 'egg']",True,[],78.0,rich,"['condiment', 'vegetable', 'vegetable', 'condi...",False,False
3,Three-Bean Tacos,"['olive oil', 'onion', 'red bell pepper', 'gre...",Hard,Quick,Cheap,Boil,"[""1 teaspoon olive oil"", ""1 cup diced onion"", ...","[""Heat oil in a large skillet over medium-high...",22.0,Medium (20-40 mins),"['olive oil', 'onion', 'red bell pepper', 'gre...",False,['cheddar cheese'],True,[],87.5,rich,"['condiment', 'vegetable', 'vegetable', 'veget...",False,False
4,Hearty Hamburger Soup,"['lean ground beef', 'white onion', 'ground bl...",Hard,Quick,Expensive,Boil,"[""900.0 gs lean ground beef"", ""1 white onion, ...","[""Brown ground beef and onion in a large pot. ...",45.0,Slow (40-90 mins),"['lean ground beef', 'white onion', 'ground bl...",False,"['lean ground beef', 'parmesan cheese', 'sour ...",False,['lean ground beef'],96.3,rich,"['meat', 'vegetable', 'condiment', 'spice/herb...",False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44618,California Chili Powder Recipe,"['Mexico chilies', 'grnd cumin', 'cayenne', 'o...",Easy,Average,Cheap,Other,"[""0.25 c. grnd Calif or possibly New Mexico ch...","[""In a 0.3333333333- to 0.5-c. tall, narrow cl...",30.0,Medium (20-40 mins),"['Mexico chilies', 'grnd cumin', 'cayenne', 'o...",True,[],True,[],78.8,rich,"[None, 'spice/herb', 'spice/herb', 'spice/herb...",True,True
44619,Pizza Margherita,"['batch pizza', 'marinara sauce', 'mozzarella'...",Easy,Quick,Cheap,Bake,"[""1 batch pizza dough (see preceding recipe)"",...","[""Preheat oven to 450 degrees F. Place a pizza...",10.0,Fast (10-20 mins),"['batch pizza', 'marinara sauce', 'mozzarella'...",True,[],True,[],18.0,very cheap,"['processed', 'condiment', 'dairy', None]",False,True
44620,Frenchish Chicken And Red Wine Casserole,"['tomatoes', 'red wine', 'chicken stock', 'gar...",Hard,Slow,Cheap,Boil,"[""1 (14 ounce) can chopped tomatoes"", ""1.25 cu...","[""Place the tomatoes, wine, stock, garlic, ros...",100.0,Very slow (90+ mins),"['tomatoes', 'red wine', 'chicken stock', 'gar...",False,"['chicken stock', 'chicken thighs', 'chicken']",False,"['chicken stock', 'chicken thighs', 'chicken']",60.3,expensive,"['vegetable', 'processed', 'processed', 'veget...",False,False
44621,Blueberry Almond Streusel Muffins Recipe,"['flour', 'sugar', 'baking pwdr', 'baking soda...",Hard,Average,Cheap,Bake,"[""2.5 c. all-purpose flour"", ""1 c. granulated ...","[""Preheat oven to 400 ."", ""Combine first 5 ing...",18.0,Fast (10-20 mins),"['flour', 'sugar', 'baking pwdr', 'baking soda...",False,"['milk', 'low-

In [9]:
import re
from fractions import Fraction

def convert_fractions(text):
    """Defining a method that convert fractions"""
    # Match optional whole number + fraction, or just fraction
    pattern = r'(?:(\d+)\s+)?(\d+)/(\d+)'

    def replacer(match):
        whole = int(match.group(1)) if match.group(1) else 0
        numerator = int(match.group(2))
        denominator = int(match.group(3))
        decimal = whole + Fraction(numerator, denominator)
        return "{:.10g}".format(float(decimal))  # Clean, no trailing zeros

    return re.sub(pattern, replacer, text)

In [10]:
'''
from fractions import Fraction

# Fixed conversions
CONVERSIONS = {
    "oz": 30,      # ounce → gram
    "lb": 450,      # pounds → gram
    "pt": 475,      # pint → milliliter
    "qt": 950,      # quart → milliliter
    "inch": 2.5        # inches → centimeter
}

def convert_units(text):
    """Defining a method that converts units by using the fixed conversions"""
    if pd.isna(text):
        return text

    # Convert oz, lb, pt, qt (form: "4 oz", "2.5 lb", etc.)
    for unit, factor in CONVERSIONS.items():
        pattern = r'(\d+(\.\d+)?)\s*' + unit
        text = re.sub(pattern, lambda m: f"{round(float(m.group(1)) * factor, 1)} {unit_to_metric(unit)}", text)

    # Convert measure in inches in pans
    text = re.sub(r'(\d+)\s*x\s*(\d+)[-\s]*inch', lambda
        m: f"{int(m.group(1)) * CONVERSIONS['inch']:.0f} x {int(m.group(2)) * CONVERSIONS['inch']:.0f} cm", text)

    return text

def convert_fahrenheit_to_celsius(text):
    # Match degrees like 275°, 275 °F, 275\u00b0F (escaped), etc.
    pattern = r'(\d+)\s*(?:°|\\u00b0)'

    def replacer(match):
        fahrenheit = int(match.group(1))
        celsius = round((fahrenheit - 32) * 5 / 9)
        return f"{celsius}°C"

    return re.sub(pattern, replacer, text)

def unit_to_metric(unit):
    return {
        "oz": "g",
        "lb": "g",
        "pt": "ml",
        "qt": "ml",
        "inch": "cm"
    }[unit]

# Applying conversions on the INGREDIENTS and DIRECTIONS columns
df['INGREDIENTS'] = df['ingredients'].apply(convert_fractions)
df['DIRECTIONS'] = df['directions'].apply(convert_fractions)

df['INGREDIENTS'] = df['INGREDIENTS'].apply(convert_units)
df['DIRECTIONS'] = df['DIRECTIONS'].apply(convert_units)

df['DIRECTIONS'] = df['DIRECTIONS'].apply(convert_fahrenheit_to_celsius)

# Renaming the title column and dropping the converted columns
df = df.rename(columns={"title": "TITLE"})
df = df.drop(['ingredients', 'directions'], axis=1)

df[['TITLE', 'INGREDIENTS', 'DIRECTIONS']].head(200)
'''

KeyError: 'ingredients'

In [11]:
import ast
import re

# Patterns to detect time expressions
TIME_PATTERN = re.compile(
    r'(\d+(?:\.\d+)?)\s*(minute|minutes|min|mins|m|hour|hours|hr|hrs|h|day|days|d)\b',
    re.IGNORECASE
)

RANGE_PATTERN = re.compile(
    r'(\d+)\s*-\s*(\d+)\s*(minute|minutes|min|mins|m|hour|hours|hr|hrs|h)\b',
    re.IGNORECASE
)

# Time estimates for common preparation methods
PREP_ESTIMATES = {
    'bake': 45, 'boil': 20, 'fry': 15, 'grill': 25, 'chill': 120,
    'simmer': 30, 'marinate': 60, 'microwave': 10, 'no-bake': 20,
    'refrigerate': 180, 'freeze': 240
}

# Special cases for recipe types
RECIPE_TYPE_ESTIMATES = {
    'pasta': 10, 'salad': 15, 'cake': 45, 'pie': 60, 'stew': 120,
    'casserole': 60, 'soup': 30, 'cookies': 30, 'bread': 90,
    'fudge': 20, 'candy': 30
}

def convert_to_minutes(qty, unit):
    """Converts time quantity to minutes."""
    qty = float(qty)
    unit = unit.lower()
    if unit.startswith('m'): return qty
    if unit.startswith('h'): return qty * 60
    if unit.startswith('d'): return qty * 1440
    return 0

def estimate_by_recipe_type(title, ingredients):
    """Fallback estimation based on recipe title or ingredients."""
    title = str(title).lower()
    ingredients = str(ingredients).lower()

    # Check for specific recipe types
    for keyword, est in RECIPE_TYPE_ESTIMATES.items():
        if keyword in title:
            return est

    if any(word in ingredients for word in ['raw', 'fresh']):
        return 15
    if 'frozen' in ingredients:
        return 20
    if 'canned' in ingredients:
        return 10

    # Default fallback
    return 30

def clean_instruction_step(step):
    """Normalize range formats and identify special cases."""
    # Replace ranges (e.g., "10-15 minutes" → "15 minutes")
    step = RANGE_PATTERN.sub(lambda m: f"{m.group(2)} {m.group(3)}", step)

    # Handle special keywords
    lowered = step.lower()
    if 'overnight' in lowered:
        return 480
    if 'until set' in lowered or 'until firm' in lowered:
        return 60

    # Sum all time expressions
    return sum(
        convert_to_minutes(qty, unit)
        for qty, unit in TIME_PATTERN.findall(step)
    )

def parse_instructions(instructions):
    """Parses instructions and extracts total estimated time in minutes."""
    total_time = 0

    if not isinstance(instructions, list):
        try:
            instructions = ast.literal_eval(str(instructions))
        except:
            instructions = [instructions]

    for step in instructions:
        if isinstance(step, str):
            total_time += clean_instruction_step(step)

    return total_time

def categorize_time(total_time):
    """Classify total time into labeled categories."""
    if total_time == 0:
        return 'Not specified'
    if total_time < 10:
        return 'Very fast (0-10 mins)'
    if total_time < 20:
        return 'Fast (10-20 mins)'
    if total_time < 40:
        return 'Medium (20-40 mins)'
    if total_time < 90:
        return 'Slow (40-90 mins)'
    return 'Very slow (90+ mins)'

# Calculate total preparation time
df['total_time'] = df['DIRECTIONS'].apply(parse_instructions)

# Estimate time for unspecified recipes
missing = df['total_time'] == 0
df.loc[missing, 'total_time'] = df[missing].apply(
    lambda row: estimate_by_recipe_type(row['TITLE'], row['INGREDIENTS']),
    axis=1
)

# Categorize preparation times
df['PREPARATION_TIME'] = df['total_time'].apply(categorize_time)

df[['TITLE', 'INGREDIENTS', 'DIRECTIONS', 'PREPARATION_TIME']].head(200)

,TITLE,INGREDIENTS,DIRECTIONS,PREPARATION_TIME
0,Western Sizzlin Bread Pudding,"[""16 eggs, beaten"", ""6 -8 evaporated milk, 360...","[""Distribute rolls and cinnamon roll in (4) 2\...",Very slow (90+ mins)
1,Creamy Tuna And Bacon Salad (Reduced Fat),"[""1 (6 ounce) can tuna, drained and flaked"", ""...","[""Put everything in a bowl and mix together un...",Fast (10-20 mins)
2,Spinach And Mushroom Lasagna,"[""1 tablespoon olive oil"", ""1 medium onion , c...","[""Heat oven to 375 degrees."", """", ""In medium s...",Slow (40-90 mins)
3,Three-Bean Tacos,"[""1 teaspoon olive oil"", ""1 cup diced onion"", ...","[""Heat oil in a large skillet over medium-high...",Medium (20-40 mins)
4,Hearty Hamburger Soup,"[""900.0 gs lean ground beef"", ""1 white onion, ...","[""Brown ground beef and onion in a large pot. ...",Slow (40-90 mins)
...,...,...,...,...
195,Sweet Potato Brotchen (Bread Rolls),"[""450 g sweet potatoes"", ""15 g butter or olive...","[""Peel and cut sweet potatoes into chunks."", ""...",Very slow (90+ mins)
196,Green Cleaner and Bug Repellant in One (Concen...,"[""1 tablespoon essential oil (part)"", ""3 table...","[""Mix all ingredients."", ""Store in a glass jar...",Medium (20-40 mins)
197,Chocolate Chip Pie,"[""1 graham cracker crust"", ""20 large marshmall...","[""Make crust (I buy the prepared ones from the...",Slow (40-90 mins)
198,Poppy Seed Dressing,"[""0.75 c. sugar"", ""2 tsp. dry mustard"", ""2 tsp...","[""Combine first 6 ingredients in a blender con...",Very fast (0-10 mins)


In [69]:
'''
# Conversione unità → kg
UNIT_CONVERSION = {
    'cup': 0.24, 'c.': 0.24, 'c': 0.24,
    'teaspoon': 0.005, 'tsp': 0.005, 'tsp.': 0.005,
    'tablespoon': 0.015, 'tbsp': 0.015, 'tbsp.': 0.015,
    'package': 0.2, 'pkg': 0.2, 'pkg.': 0.2,
    'can': 0.4, 'carton': 1.0,
    'g': 0.001, 'g.': 0.001, 'gram': 0.001,
    'kg': 1.0, 'kilogram': 1.0,
    'lb': 0.4536, 'pound': 0.4536,
    'oz': 0.02835, 'ounce': 0.02835,
    'large package': 0.5, 'large pkg.': 0.5, 'large pkg': 0.5
}

# 3. Parsing function (qty, unit, name, grams_in_paren)
def parse_ingredient(ing_str):
    # peso fra parentesi
    m = re.search(r'\(\s*([0-9]+(?:\.[0-9]+)?)\s*g\.?\s*\)', ing_str, re.IGNORECASE)
    grams = float(m.group(1)) if m else None

    # quantity e unit
    patt = r'^\s*([\d\/.\s]+)?\s*([a-zA-Z\.]+)?'
    m2 = re.match(patt, ing_str)
    qty_raw = m2.group(1) if m2 else None
    try:
        qty = eval(qty_raw.replace(' ', '+')) if qty_raw else 1.0
    except:
        qty = 1.0
    unit = (m2.group(2) or '').strip('.').lower() if m2 else ''

    # name pulito
    name = re.sub(r'\(.*?\)|optional', '', ing_str, flags=re.IGNORECASE)
    # rimuovo quantità/unità
    name = re.sub(r'^[\d\/.\s]+\s*[a-zA-Z\.]*', '', name).strip().lower()
    return qty, unit, name, grams

# 4. Calcolo posizionale con fallback substring e token
def calculate_recipe_cost_positional(ingredients_list, ner_list):
    total = 0.0
    missing = []

    for ing_str, ner_item in zip(ingredients_list, ner_list):
        key = ner_item.lower()
        qty, unit, name, grams = parse_ingredient(ing_str)

        # scorporo se passo i grams
        if grams is not None:
            kg = (grams / 1000) * qty
        elif unit == '':  # a pezzo
            # prendo prezzo direct key
            price = price_dict.get(key)
            # fallback substring/token
            if price is None:
                # substring match
                for k, v in price_dict.items():
                    if k in key or key in k:
                        price = v
                        break
                # token match
                if price is None:
                    for token in key.split():
                        if token in price_dict:
                            price = price_dict[token]
                            break
            if price:
                total += qty * price
            else:
                missing.append(key)
            continue
        else:
            conv = UNIT_CONVERSION.get(unit, 0.0)
            if conv == 0:
                missing.append(key)
                continue
            kg = qty * conv

        # prezzo per kg
        price = price_dict.get(key)
        if price is None:
            for k, v in price_dict.items():
                if k in key or key in k:
                    price = v
                    break
        if price is None:
            missing.append(key)
            continue

        total += kg * price

    return round(total, 2), missing

# 5. Applica al DataFrame
    df['INGREDIENTS'] = df['INGREDIENTS'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    df['NER_clean']  = df['NER_clean'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

def compute_cost_row(row):
     cost, _ = calculate_recipe_cost_positional(row['INGREDIENTS'], row['NER_clean'])
     return cost

df['total_cost'] = df.apply(compute_cost_row, axis=1)

df['CATEGORY_COST'] = df['total_cost'].apply(categorize_cost)

# Mostra risultati
df[['INGREDIENTS', 'NER', 'CATEGORY_COST']].head(200)'''

"\n# Conversione unità → kg\nUNIT_CONVERSION = {\n    'cup': 0.24, 'c.': 0.24, 'c': 0.24,\n    'teaspoon': 0.005, 'tsp': 0.005, 'tsp.': 0.005,\n    'tablespoon': 0.015, 'tbsp': 0.015, 'tbsp.': 0.015,\n    'package': 0.2, 'pkg': 0.2, 'pkg.': 0.2,\n    'can': 0.4, 'carton': 1.0,\n    'g': 0.001, 'g.': 0.001, 'gram': 0.001,\n    'kg': 1.0, 'kilogram': 1.0,\n    'lb': 0.4536, 'pound': 0.4536,\n    'oz': 0.02835, 'ounce': 0.02835,\n    'large package': 0.5, 'large pkg.': 0.5, 'large pkg': 0.5\n}\n\n# 3. Parsing function (qty, unit, name, grams_in_paren)\ndef parse_ingredient(ing_str):\n    # peso fra parentesi\n    m = re.search(r'\\(\\s*([0-9]+(?:\\.[0-9]+)?)\\s*g\\.?\\s*\\)', ing_str, re.IGNORECASE)\n    grams = float(m.group(1)) if m else None\n\n    # quantity e unit\n    patt = r'^\\s*([\\d\\/.\\s]+)?\\s*([a-zA-Z\\.]+)?'\n    m2 = re.match(patt, ing_str)\n    qty_raw = m2.group(1) if m2 else None\n    try:\n        qty = eval(qty_raw.replace(' ', '+')) if qty_raw else 1.0\n    except:\

In [12]:
import pandas as pd
import ast
import re

df['NER_clean'] = df['NER'].apply(ast.literal_eval)

# List containing keywords
NON_VEGAN_KEYWORDS = {
    'milk', 'cheese', 'butter', 'cream', 'yogurt', 'gelatin', 'lard', 'honey',
    'egg', 'eggs', 'fish', 'meat', 'chicken', 'beef', 'pork', 'gelatina',
    'collagen', 'casein', 'whey', 'lactose', 'ghee', 'isinglass', 'carmine',
    'shellac', 'albumen', 'pepsin', 'royal jelly', 'propolis', 'cocoa butter',
    'bacon', 'sour cream', 'condensed milk', 'shredded cheese', 'cheddar',
    'paraffin', 'marshmallows', 'buttermilk', 'ground beef', 'steak', 'tuna'
}

VEGAN_EXCEPTIONS = {
    'milk': {'soy', 'almond', 'oat', 'rice', 'coconut', 'cashew', 'hazelnut'},
    'cheese': {'vegan', 'nutritional yeast', 'cashew', 'tofu'},
    'meat': {'soy', 'seitan', 'tofu', 'tempeh', 'jackfruit', 'plant-based'},
    'butter': {'vegan', 'plant', 'peanut', 'almond', 'soy'},
    'cream': {'coconut', 'soy', 'oat', 'vegan'},
    'bacon': {'vegan', 'tempeh', 'coconut'}
}

VEGAN_MODIFIERS = {
    'vegan', 'vegetable', 'plant-based', 'no milk', 'no eggs',
    'dairy-free', 'without animal derivatives', 'cruelty-free', '100% vegetable'
}

# --- Utility
def parse_ingredients(ingredients_input):
    """Always returning a parsed list of ingredients"""
    if isinstance(ingredients_input, list):
        return ingredients_input
    if isinstance(ingredients_input, str) and ingredients_input.strip():
        try:
            return ast.literal_eval(ingredients_input)
        except:
            return [
                x.strip().strip('"').strip("'")
                for x in re.split(r',(?![^[]*\])', ingredients_input.strip('[]'))
                if x.strip()
            ]
    return []

def contains_vegan_exception(ingredient, keyword):
    return any(re.search(rf'\b{re.escape(ex)}\b', ingredient, re.IGNORECASE)
               for ex in VEGAN_EXCEPTIONS.get(keyword, set()))

# --- Classification
def classify_vegan(ingredients_input):
    """Returning True or False based on the ingredients"""
    ingredients = parse_ingredients(ingredients_input)
    for ing in ingredients:
        if not isinstance(ing, str):
            continue
        ing_lower = ing.lower()
        if any(skip in ing_lower for skip in ('water', 'salt')):
            continue
        if any(re.search(rf'\b{re.escape(mod)}\b', ing_lower) for mod in VEGAN_MODIFIERS):
            continue
        for keyword in NON_VEGAN_KEYWORDS:
            if re.search(rf'\b{re.escape(keyword)}(?:s)?\b', ing_lower):
                if not contains_vegan_exception(ing_lower, keyword):
                    return False
    return True

def get_non_vegan_ingredients(ingredients_input):
    """Returning list of ingredients without vegan exceptions"""
    ingredients = parse_ingredients(ingredients_input)
    non_vegan = []
    for ing in ingredients:
        if not isinstance(ing, str):
            continue
        ing_lower = ing.lower()
        if any(skip in ing_lower for skip in ('water', 'salt')):
            continue
        for keyword in NON_VEGAN_KEYWORDS:
            if re.search(rf'\b{re.escape(keyword)}(?:s)?\b', ing_lower):
                if not contains_vegan_exception(ing_lower, keyword):
                    non_vegan.append(ing)
                break
    return non_vegan

# --- Apply
df['VEGAN'] = df['NER_clean'].apply(classify_vegan)
df['NON_VEGAN_INGREDIENTS'] = df['NER_clean'].apply(get_non_vegan_ingredients)

# --- Final validation
anti_patterns = re.compile(
    r'\b(not vegan|meat|steak|fish|cheese|egg|eggs|beef|chicken|pork|bacon|cream)\b',
    re.IGNORECASE
)

df['VEGAN'] = df.apply(
    lambda row: False if row['VEGAN'] and isinstance(row['TITLE'], str) and anti_patterns.search(row['TITLE']) else row['VEGAN'],
    axis=1
)

# View results
df[['TITLE', 'NER', 'VEGAN', 'NON_VEGAN_INGREDIENTS']].head(200)

,TITLE,NER,VEGAN,NON_VEGAN_INGREDIENTS
0,Western Sizzlin Bread Pudding,"['eggs', 'milk', 'sugar', 'vanilla', 'gallon w...",False,"[eggs, milk]"
1,Creamy Tuna And Bacon Salad (Reduced Fat),"['tuna', 'mayonnaise', 'low-fat sour cream', '...",False,"[tuna, low-fat sour cream, bacon bits]"
2,Spinach And Mushroom Lasagna,"['olive oil', 'onion', 'garlic', 'salt', 'grou...",False,"[ricotta cheese, Parmesan cheese, egg]"
3,Three-Bean Tacos,"['olive oil', 'onion', 'red bell pepper', 'gre...",False,[cheddar cheese]
4,Hearty Hamburger Soup,"['lean ground beef', 'white onion', 'ground bl...",False,"[lean ground beef, parmesan cheese, sour cream]"
...,...,...,...,...
195,Sweet Potato Brotchen (Bread Rolls),"['sweet potatoes', 'butter', 'salt', 'flour', ...",False,[butter]
196,Green Cleaner and Bug Repellant in One (Concen...,"['essential oil', 'vodka', 'liquid soap']",True,[]
197,Chocolate Chip Pie,"['graham cracker crust', 'marshmallows', 'milk...",False,"[marshmallows, milk, whipping cream]"
198,Poppy Seed Dressing,"['sugar', 'dry mustard', 'salt', 'vinegar', 'o...",True,[]


In [13]:
# List of non vegetarian ingredients
NON_VEGETARIAN_KEYWORDS = {
    'meat', 'chicken', 'beef', 'pork', 'fish', 'anchovy', 'tuna', 'salmon',
    'shellfish', 'shrimp', 'crab', 'lobster', 'bacon', 'gelatin', 'lard',
    'collagen', 'isinglass', 'pepsin', 'ground beef', 'steak', 'tuna'
}

# Exceptions for vegetarian subs
VEGETARIAN_EXCEPTIONS = {
    'meat': {'soy', 'seitan', 'tofu', 'tempeh', 'jackfruit', 'plant-based'},
    'bacon': {'vegan', 'tempeh', 'coconut'},
    'gelatin': {'agar', 'pectin'},
    'fish': {'banana blossom', 'tofu', 'plant-based'}
}

VEGETARIAN_MODIFIERS = {
    'vegetarian', 'veggie', 'plant-based', 'meatless', 'no meat',
    'without meat', 'cruelty-free'
}

# --- Utility
def parse_ingredients(ingredients_input):
    """Always returning a parsed list of ingredients"""
    if isinstance(ingredients_input, list):
        return ingredients_input
    if isinstance(ingredients_input, str) and ingredients_input.strip():
        try:
            return ast.literal_eval(ingredients_input)
        except:
            return [
                x.strip().strip('"').strip("'")
                for x in re.split(r',(?![^[]*\])', ingredients_input.strip('[]'))
                if x.strip()
            ]
    return []

def contains_vegetarian_exception(ingredient, keyword):
    exceptions = VEGETARIAN_EXCEPTIONS.get(keyword, set())
    return any(re.search(rf'\b{re.escape(ex)}\b', ingredient, re.IGNORECASE) for ex in exceptions)

# --- Classification
def classify_vegetarian(ingredients_input):
    """Returning True or False based on the ingredients"""
    ingredients = parse_ingredients(ingredients_input)
    for ing in ingredients:
        if not isinstance(ing, str):
            continue
        ing_lower = ing.lower()
        if any(skip in ing_lower for skip in ('water', 'salt')):
            continue
        if any(re.search(rf'\b{re.escape(mod)}\b', ing_lower) for mod in VEGETARIAN_MODIFIERS):
            continue
        for keyword in NON_VEGETARIAN_KEYWORDS:
            if re.search(rf'\b{re.escape(keyword)}(?:s)?\b', ing_lower):
                if not contains_vegetarian_exception(ing_lower, keyword):
                    return False
    return True

def get_non_vegetarian_ingredients(ingredients_input):
    """Returning list of ingredients without vegetarian exceptions"""
    ingredients = parse_ingredients(ingredients_input)
    non_vegetarian = []
    for ing in ingredients:
        if not isinstance(ing, str):
            continue
        ing_lower = ing.lower()
        if any(skip in ing_lower for skip in ('water', 'salt')):
            continue
        for keyword in NON_VEGETARIAN_KEYWORDS:
            if re.search(rf'\b{re.escape(keyword)}(?:s)?\b', ing_lower):
                if not contains_vegetarian_exception(ing_lower, keyword):
                    non_vegetarian.append(ing)
                break
    return non_vegetarian

# Apply to dataframe
df['VEGETARIAN'] = df['NER_clean'].apply(classify_vegetarian)
df['NON_VEGETARIAN_INGREDIENTS'] = df['NER_clean'].apply(get_non_vegetarian_ingredients)

anti_patterns = re.compile(
    r'\b(not vegetarian|not veg|meat|steak|fish|beef|chicken|pork|bacon)\b',
    re.IGNORECASE
)

df['VEGETARIAN'] = df.apply(
    lambda row: False if row['VEGETARIAN'] and isinstance(row['TITLE'], str) and anti_patterns.search(row['TITLE']) else row['VEGETARIAN'],
    axis=1
)

# Show results
df[['TITLE', 'NER', 'VEGETARIAN', 'NON_VEGETARIAN_INGREDIENTS']].head(200)

,TITLE,NER,VEGETARIAN,NON_VEGETARIAN_INGREDIENTS
0,Western Sizzlin Bread Pudding,"['eggs', 'milk', 'sugar', 'vanilla', 'gallon w...",True,[]
1,Creamy Tuna And Bacon Salad (Reduced Fat),"['tuna', 'mayonnaise', 'low-fat sour cream', '...",False,"[tuna, bacon bits]"
2,Spinach And Mushroom Lasagna,"['olive oil', 'onion', 'garlic', 'salt', 'grou...",True,[]
3,Three-Bean Tacos,"['olive oil', 'onion', 'red bell pepper', 'gre...",True,[]
4,Hearty Hamburger Soup,"['lean ground beef', 'white onion', 'ground bl...",False,[lean ground beef]
...,...,...,...,...
195,Sweet Potato Brotchen (Bread Rolls),"['sweet potatoes', 'butter', 'salt', 'flour', ...",True,[]
196,Green Cleaner and Bug Repellant in One (Concen...,"['essential oil', 'vodka', 'liquid soap']",True,[]
197,Chocolate Chip Pie,"['graham cracker crust', 'marshmallows', 'milk...",True,[]
198,Poppy Seed Dressing,"['sugar', 'dry mustard', 'salt', 'vinegar', 'o...",True,[]


Qui inizia la magia:

In [72]:
import numpy as np
import torch
from transformers import pipeline
import re
from tqdm import tqdm

gpu = torch.cuda.get_device_name(0)
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA device count: {torch.cuda.device_count()}")
print(f"Current device: {torch.cuda.current_device()}")
print(f"Device name: {gpu}")

# Eseguire questa cella solo se runnata dal Barone
if gpu == "NVIDIA GeForce RTX 3060 Ti":
    classifier = pipeline(
        task="zero-shot-classification",
        model="MoritzLaurer/deberta-v3-base-zeroshot-v1.1-all-33",
        device=0,
        torch_dtype=torch.float16,
        model_kwargs={"cache_dir": "./cache"},
        batch_size=32,  # Optimal for RTX 3060 Ti
        framework="pt"
    )
    
    # %% Data Loading
    df['NER'] = df['NER'].apply(eval)  # Convert string lists to actual lists
    
    # %% Food Categories & Prices (EUR/kg)
    CATEGORIES = [
        "dairy", "meat", "seafood", "grain", "vegetable",
        "fruit", "spice/herb", "processed", "sweetener",
        "condiment", "legume", "oil/fat"
    ]
    
    MEDIAN_PRICES = {
        "dairy": 3.50,       # Milk, cheese
        "meat": 7.50,        # Chicken, beef
        "seafood": 12.00,    # Fish, shrimp
        "grain": 2.20,       # Flour, rice
        "vegetable": 1.80,   # Onions, garlic
        "fruit": 2.50,       # Tomatoes, bananas
        "spice/herb": 18.00, # Vanilla, cinnamon
        "processed": 4.50,   # Pasta, canned goods
        "sweetener": 2.20,   # Sugar
        "condiment": 5.00,   # Mayo, dressings
        "legume": 3.00,      # Beans, lentils
        "oil/fat": 8.00      # Olive oil
    }
    
    # %% Ingredient Cleaning (Fixed)
    def clean_ingredient(ingredient: str) -> str:
        """Conservative cleaning preserving ingredient names"""
        # Remove quantities (e.g., "200g", "1/2 cup")
        cleaned = re.sub(r'\b\d+[\d/\.]*\s*[a-z]*\b', '', ingredient, flags=re.IGNORECASE)
        # Remove special chars except spaces
        cleaned = re.sub(r'[^\w\s]', '', cleaned).strip().lower()
        return cleaned if cleaned else "unknown"
    
    # %% Batch Classification (GPU-optimized)
    classification_cache = {}
    
    def batch_classify(ingredients: list, batch_size: int = 16) -> dict:  # Reduced batch size
        unique_ingredients = list(set(ingredients))
    
        with torch.no_grad():  # Disable gradient tracking
            for batch in tqdm([unique_ingredients[i:i+batch_size]
                               for i in range(0, len(unique_ingredients), batch_size)],
                              desc="Classifying Ingredients"):
                # Process batch on GPU
                results = classifier(batch, CATEGORIES, multi_label=False)
    
                # Cache results
                for ing, result in zip(batch, results):
                    classification_cache[ing] = result['labels'][0]
    
                # Clear GPU cache
                torch.cuda.empty_cache()
    
        return classification_cache
    
    # %% Process Entire Dataset
    # Get all unique ingredients
    all_ingredients = [clean_ingredient(ing)
                       for recipe in df['NER']
                       for ing in recipe]
    unique_ingredients = list(set(all_ingredients))
    
    # Batch classify with progress bar
    _ = batch_classify(unique_ingredients, batch_size=64)
    
    # Calculate recipe costs
    df['total_cost'] = df['NER'].apply(
        lambda x: round(sum(
            MEDIAN_PRICES.get(classification_cache[clean_ingredient(ing)], 3.00)
            for ing in x
        ), 2)
    )
    
    # Dynamic price categorization
    costs = df['total_cost'].values
    
    # threshold per le categorie degli prezzi
    very_cheap, cheap, medium, expensive = np.percentile(costs, [20, 40, 60, 85])
    
    df['price_tag'] = df['total_cost'].apply(
        lambda x: 'very cheap' if x <= very_cheap
        else 'cheap' if x <= cheap
        else 'medium' if x <= medium
        else 'expensive' if x <= expensive
        else 'rich'
    )

    def get_categories(ingredient_list):
        return [classification_cache.get(ing) for ing in ingredient_list]
    
    def is_vegan(cat_list):
        return all(cat not in ['dairy', 'meat', 'seafood'] for cat in cat_list if cat is not None)
    
    def is_vegetarian(cat_list):
        return all(cat not in ['meat', 'seafood'] for cat in cat_list if cat is not None)
    
    df['categories'] = df['NER_clean'].apply(get_categories)
    df['vegan'] = df['categories'].apply(is_vegan)
    df['vegetarian'] = df['categories'].apply(is_vegetarian)
    
    # Save outputs
    df.to_csv('full_tagged_dataset_10%', index=False)

PyTorch version: 2.7.0+cu118
CUDA available: True
CUDA device count: 1
Current device: 0
Device name: NVIDIA GeForce RTX 3060 Ti


Device set to use cuda:0
Classifying Ingredients: 100%|██████████| 260/260 [02:41<00:00,  1.61it/s]


Qui la magia finisce :)

# Creazione del modello per la classificazione del tag 'PREPARATION_TIME' e testing

In [15]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import pickle

# Default parameters
MAX_LEN = 500
NUM_WORDS = 20000
BATCH_SIZE = 32
EPOCHS = 10

# Method to preprocess a dataframe
def preprocess(df, text_columns, label_column, n=10000):
    df = df[text_columns + [label_column]].dropna().head(n).copy()
    df[text_columns[0]] = df[text_columns[0]].apply(eval)
    df[text_columns[1]] = df[text_columns[1]].apply(eval)
    df["full_text"] = df[text_columns[0]].apply(lambda x: " ".join(x)) + " " + df[text_columns[1]].apply(lambda x: " ".join(x))

    label_encoder = LabelEncoder()
    df["encoded_label"] = label_encoder.fit_transform(df[label_column])

    return df, label_encoder

# Method to train a classificator
def train_text_classifier(df, label_encoder, model_name_prefix):
    X = df["full_text"]
    y = df["encoded_label"]

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

    tokenizer = Tokenizer(num_words=NUM_WORDS, oov_token="<OOV>")
    tokenizer.fit_on_texts(X_train)

    X_train_seq = tokenizer.texts_to_sequences(X_train)
    X_val_seq = tokenizer.texts_to_sequences(X_val)

    X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding="post", truncating="post")
    X_val_pad = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding="post", truncating="post")

    model = Sequential([
        Embedding(input_dim=NUM_WORDS, output_dim=128, input_length=MAX_LEN),
        GlobalMaxPooling1D(),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dense(len(label_encoder.classes_), activation='softmax')
    ])

    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.fit(
        X_train_pad, y_train,
        validation_data=(X_val_pad, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=[early_stop]
    )

    # Saving model
    model.save(f"classifiers/prep_time/{model_name_prefix}.h5")
    with open(f"classifiers/prep_time/{model_name_prefix}_label_mapping.pkl", "wb") as f:
        pickle.dump(dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))), f)

    with open(f"classifiers/prep_time/{model_name_prefix}_tokenizer.pkl", "wb") as f:
        pickle.dump(tokenizer, f)

    print(f"Modello '{model_name_prefix}' salvato con successo.")
    return model

# === FIRST MODEL: PREPARATION_TIME ===
df_time = df[["INGREDIENTS", "DIRECTIONS", "PREPARATION_TIME"]].copy()
df_time, prep_time_encoder = preprocess(df_time, ["INGREDIENTS", "DIRECTIONS"], "PREPARATION_TIME")
prep_time_model = train_text_classifier(df_time, prep_time_encoder, model_name_prefix="prep_time_classifier")

C:\Users\aless\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 11s 40ms/step - accuracy: 0.3440 - loss: 1.5039 - val_accuracy: 0.5595 - val_loss: 1.0874
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - accuracy: 0.5825 - loss: 1.0628 - val_accuracy: 0.6415 - val_loss: 0.9110
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 34ms/step - accuracy: 0.6505 - loss: 0.9110 - val_accuracy: 0.6780 - val_loss: 0.8400
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 34ms/step - accuracy: 0.6937 - loss: 0.8323 - val_accuracy: 0.6880 - val_loss: 0.8132
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - accuracy: 0.7184 - loss: 0.7549 - val_accuracy: 0.6755 - val_loss: 0.8112
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 30ms/step - accuracy: 0.7518 - loss: 0.6918 - val_accuracy: 0.6895 - val_loss: 0.8040
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/step - accuracy: 0.7634 - loss: 0.6626 - val_accuracy: 0.6875 - val_loss: 0.8012
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - accuracy: 0.7805 - loss: 0.6040 - val_acc

Modello 'prep_time_classifier' salvato con successo.


In [16]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

MAX_LEN = 500  # same as training

# --- Load model and tokenizer ---
prep_time_model = load_model("classifiers/prep_time/prep_time_classifier.h5")

with open("classifiers/prep_time/prep_time_classifier_tokenizer.pkl", "rb") as f:
    prep_time_tokenizer = pickle.load(f)

with open("classifiers/prep_time/prep_time_classifier_label_mapping.pkl", "rb") as f:
    prep_time_label_map = pickle.load(f)

# Reverse dictionaries to decode predictions
inv_prep_time_label_map = {v: k for k, v in prep_time_label_map.items()}

# List of 10 recipes with expected time
recipes = [
    {
        "ingredients": "200g spaghetti, 2 cloves garlic, chili pepper, extra virgin olive oil, salt",
        "instructions": "Cook the spaghetti. Meanwhile, sauté garlic and chili in the oil. Mix with the pasta.",
        "expected_time": "15 min"
    },
    {
        "ingredients": "Lettuce, 1 can of tuna, tomatoes, black olives, onion, olive oil, salt",
        "instructions": "Wash and cut the vegetables. Add the drained tuna and olives. Dress with oil and salt.",
        "expected_time": "10 min"
    },
    {
        "ingredients": "300g Carnaroli rice, 1 saffron sachet, vegetable broth, butter, onion, Parmesan cheese",
        "instructions": "Sauté the onion, add rice and deglaze with wine. Gradually pour in the broth. Add saffron and stir in butter and cheese.",
        "expected_time": "30 min"
    },
    {
        "ingredients": "3 eggs, salt, pepper, butter",
        "instructions": "Beat the eggs with salt and pepper. Pour into a pan with melted butter and stir until set.",
        "expected_time": "5 min"
    },
    {
        "ingredients": "Lasagna sheets, meat sauce, béchamel, Parmesan cheese",
        "instructions": "Layer pasta, sauce, béchamel and cheese. Bake at 180°C for 40 minutes.",
        "expected_time": "60 min"
    },
    {
        "ingredients": "Apples, bananas, oranges, strawberries, lemon juice, sugar",
        "instructions": "Cut the fruit into pieces, mix with lemon juice and sugar, refrigerate.",
        "expected_time": "10 min"
    },
    {
        "ingredients": "Chicken breast, onion, curry powder, cream or coconut milk, oil, salt",
        "instructions": "Brown the onion, add diced chicken, curry, and cream. Cook until the chicken is tender.",
        "expected_time": "25 min"
    },
    {
        "ingredients": "Ladyfingers, mascarpone, eggs, sugar, coffee, cocoa powder",
        "instructions": "Make a cream with yolks, sugar and mascarpone. Layer ladyfingers soaked in coffee and the cream. Dust with cocoa and chill.",
        "expected_time": "30 min"
    },
    {
        "ingredients": "2 eggs, 250ml milk, 125g flour, sugar, butter",
        "instructions": "Mix flour, eggs and milk into a batter. Cook in pan with butter. Fill as desired.",
        "expected_time": "20 min"
    },
    {
        "ingredients": "Mixed dried legumes, onion, carrot, celery, vegetable broth, oil, salt",
        "instructions": "Soak legumes for 12 hours. Sauté onion, carrot, celery. Add legumes and broth and cook for at least an hour.",
        "expected_time": "90 min"
    }
]

# Prepare input texts for models (ingredients + instructions)
texts = [r["ingredients"] + " " + r["instructions"] for r in recipes]

# Tokenize and padding
prep_sequences = prep_time_tokenizer.texts_to_sequences(texts)
prep_padded = pad_sequences(prep_sequences, maxlen=MAX_LEN, padding='post', truncating='post')

# Predicting time with probabilities
prep_preds = prep_time_model.predict(prep_padded)
prep_classes = prep_preds.argmax(axis=1)
prep_labels = [inv_prep_time_label_map[c] for c in prep_classes]

# Print results with probabilities
for i, recipe in enumerate(recipes):
    print(f"Recipe {i+1}:")
    print(f"  Expected preparation time: {recipe['expected_time']} minutes")
    print(f"  Predicted preparation time: {prep_labels[i]}")
    prep_probs_str = ", ".join([f"{inv_prep_time_label_map[j]}: {prep_preds[i][j]*100:.2f}%" for j in range(len(prep_preds[i]))])
    print(f"  Preparation time probabilities: {prep_probs_str}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
Recipe 1:
  Expected preparation time: 15 min minutes
  Predicted preparation time: Fast (10-20 mins)
  Preparation time probabilities: Fast (10-20 mins): 47.13%, Medium (20-40 mins): 32.66%, Slow (40-90 mins): 5.50%, Very fast (0-10 mins): 12.92%, Very slow (90+ mins): 1.79%
Recipe 2:
  Expected preparation time: 10 min minutes
  Predicted preparation time: Fast (10-20 mins)
  Preparation time probabilities: Fast (10-20 mins): 68.13%, Medium (20-40 mins): 23.42%, Slow (40-90 mins): 1.01%, Very fast (0-10 mins): 7.07%, Very slow (90+ mins): 0.36%
Recipe 3:
  Expected preparation time: 30 min minutes
  Predicted preparation time: Medium (20-40 mins)
  Preparation time probabilities: Fast (10-20 mins): 13.88%, Medium (20-40 mins): 62.13%, Slow (40-90 mins): 14.24%, Very fast (0-10 mins): 7.64%, Very slow (90+ mins): 2.11%
Recipe 4:
  Expected preparation time: 5 min minutes
  Predicted preparation time: Medium (20-40 mins)
  Preparation time probabil

# Creazione del modello per la classificazione del tag 'DIFFICULTY' e testing

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import pickle

# Default parameters
MAX_LEN = 500
NUM_WORDS = 20000
BATCH_SIZE = 32
EPOCHS = 10

def preprocess(df, text_columns, label_column, n=10000):
    df = df[text_columns + [label_column]].dropna().head(n).copy()
    df[text_columns[0]] = df[text_columns[0]].apply(eval)
    df[text_columns[1]] = df[text_columns[1]].apply(eval)
    df["full_text"] = df[text_columns[0]].apply(lambda x: " ".join(x)) + " " + df[text_columns[1]].apply(lambda x: " ".join(x))

    label_encoder = LabelEncoder()
    df["encoded_label"] = label_encoder.fit_transform(df[label_column])

    return df, label_encoder

def train_text_classifier(df, label_encoder, model_name_prefix):
    X = df["full_text"]
    y = df["encoded_label"]

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

    tokenizer = Tokenizer(num_words=NUM_WORDS, oov_token="<OOV>")
    tokenizer.fit_on_texts(X_train)

    X_train_seq = tokenizer.texts_to_sequences(X_train)
    X_val_seq = tokenizer.texts_to_sequences(X_val)

    X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding="post", truncating="post")
    X_val_pad = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding="post", truncating="post")

    model = Sequential([
        Embedding(input_dim=NUM_WORDS, output_dim=128, input_length=MAX_LEN),
        GlobalMaxPooling1D(),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dense(len(label_encoder.classes_), activation='softmax')
    ])

    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.fit(
        X_train_pad, y_train,
        validation_data=(X_val_pad, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=[early_stop]
    )

    model.save(f"classifiers/difficulty/{model_name_prefix}.h5")
    with open(f"classifiers/difficulty/{model_name_prefix}_label_mapping.pkl", "wb") as f:
        pickle.dump(dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))), f)

    with open(f"classifiers/difficulty/{model_name_prefix}_tokenizer.pkl", "wb") as f:
        pickle.dump(tokenizer, f)

    print(f"Modello '{model_name_prefix}' salvato con successo.")
    return model

# === SECOND MODEL: DIFFICULTY ===
df_difficulty = pd.read_csv("recipes_with_prices")
df_difficulty, difficulty_encoder = preprocess(df_difficulty, ["ingredients", "directions"], "difficulty")
difficulty_model = train_text_classifier(df_difficulty, difficulty_encoder, model_name_prefix="difficulty_classifier")

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

MAX_LEN = 500  # same as training

# Load model and tokenizer
difficulty_model = load_model("classifiers/difficulty/difficulty_classifier.h5")

with open("classifiers/difficulty/difficulty_classifier_tokenizer.pkl", "rb") as f:
    difficulty_tokenizer = pickle.load(f)

with open("classifiers/difficulty/difficulty_classifier_label_mapping.pkl", "rb") as f:
    difficulty_label_map = pickle.load(f)

# Reverse dictionaries to decode predictions
inv_difficulty_label_map = {v: k for k, v in difficulty_label_map.items()}

# List of 10 recipes with expected difficulty
recipes = [
    {
        "ingredients": "200g spaghetti, 2 cloves garlic, chili pepper, extra virgin olive oil, salt",
        "instructions": "Cook the spaghetti. Meanwhile, sauté garlic and chili in the oil. Mix with the pasta.",
        "expected_difficulty": "Medium"
    },
    {
        "ingredients": "Lettuce, 1 can of tuna, tomatoes, black olives, onion, olive oil, salt",
        "instructions": "Wash and cut the vegetables. Add the drained tuna and olives. Dress with oil and salt.",
        "expected_difficulty": "Easy"
    },
    {
        "ingredients": "300g Carnaroli rice, 1 saffron sachet, vegetable broth, butter, onion, Parmesan cheese",
        "instructions": "Sauté the onion, add rice and deglaze with wine. Gradually pour in the broth. Add saffron and stir in butter and cheese.",
        "expected_difficulty": "Medium"
    },
    {
        "ingredients": "3 eggs, salt, pepper, butter",
        "instructions": "Beat the eggs with salt and pepper. Pour into a pan with melted butter and stir until set.",
        "expected_difficulty": "Easy"
    },
    {
        "ingredients": "Lasagna sheets, meat sauce, béchamel, Parmesan cheese",
        "instructions": "Layer pasta, sauce, béchamel and cheese. Bake at 180°C for 40 minutes.",
        "expected_difficulty": "Hard"
    },
    {
        "ingredients": "Apples, bananas, oranges, strawberries, lemon juice, sugar",
        "instructions": "Cut the fruit into pieces, mix with lemon juice and sugar, refrigerate.",
        "expected_difficulty": "Easy"
    },
    {
        "ingredients": "Chicken breast, onion, curry powder, cream or coconut milk, oil, salt",
        "instructions": "Brown the onion, add diced chicken, curry, and cream. Cook until the chicken is tender.",
        "expected_difficulty": "Medium"
    },
    {
        "ingredients": "Ladyfingers, mascarpone, eggs, sugar, coffee, cocoa powder",
        "instructions": "Make a cream with yolks, sugar and mascarpone. Layer ladyfingers soaked in coffee and the cream. Dust with cocoa and chill.",
        "expected_difficulty": "Medium"
    },
    {
        "ingredients": "2 eggs, 250ml milk, 125g flour, sugar, butter",
        "instructions": "Mix flour, eggs and milk into a batter. Cook in pan with butter. Fill as desired.",
        "expected_difficulty": "Medium"
    },
    {
        "ingredients": "Mixed dried legumes, onion, carrot, celery, vegetable broth, oil, salt",
        "instructions": "Soak legumes for 12 hours. Sauté onion, carrot, celery. Add legumes and broth and cook for at least an hour.",
        "expected_difficulty": "Hard"
    }
]

# Prepare input texts for models (ingredients + instructions)
texts = [r["ingredients"] + " " + r["instructions"] for r in recipes]

# Tokenize and padding
diff_sequences = difficulty_tokenizer.texts_to_sequences(texts)
diff_padded = pad_sequences(diff_sequences, maxlen=MAX_LEN, padding='post', truncating='post')

# Predicting difficulty
diff_preds = difficulty_model.predict(diff_padded)
diff_classes = diff_preds.argmax(axis=1)
diff_labels = [inv_difficulty_label_map[c] for c in diff_classes]

# Print results
for i, recipe in enumerate(recipes):
    print(f"Recipe {i+1}:")
    print(f"  Expected difficulty: {recipe['expected_difficulty']}")
    print(f"  Predicted difficulty: {diff_labels[i]}")
    diff_probs_str = ", ".join([f"{inv_difficulty_label_map[j]}: {diff_preds[i][j]*100:.2f}%" for j in range(len(diff_preds[i]))])
    print(f"  Difficulty probabilities: {diff_probs_str}\n")

# Creazione del modello per la classificazione del tag 'VEGAN' e testing

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import pickle

# Default parameters
MAX_LEN = 500
NUM_WORDS = 20000
BATCH_SIZE = 32
EPOCHS = 10

def preprocess(df, text_columns, label_column, n=10000):
    df = df[text_columns + [label_column]].dropna().head(n).copy()
    df[text_columns[0]] = df[text_columns[0]].apply(eval)
    df[text_columns[1]] = df[text_columns[1]].apply(eval)
    df["full_text"] = df[text_columns[0]].apply(lambda x: " ".join(x)) + " " + df[text_columns[1]].apply(lambda x: " ".join(x))

    label_encoder = LabelEncoder()
    df["encoded_label"] = label_encoder.fit_transform(df[label_column])

    return df, label_encoder

def train_text_classifier(df, label_encoder, model_name_prefix):
    X = df["full_text"]
    y = df["encoded_label"]

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

    tokenizer = Tokenizer(num_words=NUM_WORDS, oov_token="<OOV>")
    tokenizer.fit_on_texts(X_train)

    X_train_seq = tokenizer.texts_to_sequences(X_train)
    X_val_seq = tokenizer.texts_to_sequences(X_val)

    X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding="post", truncating="post")
    X_val_pad = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding="post", truncating="post")

    model = Sequential([
        Embedding(input_dim=NUM_WORDS, output_dim=128, input_length=MAX_LEN),
        GlobalMaxPooling1D(),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dense(len(label_encoder.classes_), activation='softmax')
    ])

    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.fit(
        X_train_pad, y_train,
        validation_data=(X_val_pad, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=[early_stop]
    )

    model.save(f"classifiers/vegan/{model_name_prefix}.h5")
    with open(f"classifiers/vegan/{model_name_prefix}_label_mapping.pkl", "wb") as f:
        pickle.dump(dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))), f)

    with open(f"classifiers/vegan/{model_name_prefix}_tokenizer.pkl", "wb") as f:
        pickle.dump(tokenizer, f)

    print(f"Modello '{model_name_prefix}' salvato con successo.")
    return model

# === THIRD MODEL: VEGAN ===
df_vegan = df[["INGREDIENTS", "DIRECTIONS", "VEGAN"]].copy()
df_vegan, vegan_encoder = preprocess(df_vegan, ["INGREDIENTS", "DIRECTIONS"], "VEGAN")
vegan_classifier_model = train_text_classifier(df_vegan, vegan_encoder, model_name_prefix="vegan_classifier")

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

MAX_LEN = 500  # same as training

# Load model and tokenizer
vegan_model = load_model("classifiers/vegan/vegan_classifier.h5")

with open("classifiers/vegan/vegan_classifier_tokenizer.pkl", "rb") as f:
    vegan_tokenizer = pickle.load(f)

with open("classifiers/vegan/vegan_classifier_label_mapping.pkl", "rb") as f:
    vegan_label_map = pickle.load(f)

# Reverse dictionaries to decode predictions
inv_vegan_label_map = {v: k for k, v in vegan_label_map.items()}

# List of 10 recipes with expected vegan
recipes = [
    {
        "ingredients": "200g spaghetti, 2 cloves garlic, chili pepper, extra virgin olive oil, salt",
        "instructions": "Cook the spaghetti. Meanwhile, sauté garlic and chili in the oil. Mix with the pasta.",
        "expected_vegan": True
    },
    {
        "ingredients": "Lettuce, 1 can of tuna, tomatoes, black olives, onion, olive oil, salt",
        "instructions": "Wash and cut the vegetables. Add the drained tuna and olives. Dress with oil and salt.",
        "expected_vegan": False
    },
    {
        "ingredients": "300g Carnaroli rice, 1 saffron sachet, vegetable broth, butter, onion, Parmesan cheese",
        "instructions": "Sauté the onion, add rice and deglaze with wine. Gradually pour in the broth. Add saffron and stir in butter and cheese.",
        "expected_vegan": False
    },
    {
        "ingredients": "3 eggs, salt, pepper, butter",
        "instructions": "Beat the eggs with salt and pepper. Pour into a pan with melted butter and stir until set.",
        "expected_vegan": False
    },
    {
        "ingredients": "Lasagna sheets, meat sauce, béchamel, Parmesan cheese",
        "instructions": "Layer pasta, sauce, béchamel and cheese. Bake at 180°C for 40 minutes.",
        "expected_vegan": False
    },
    {
        "ingredients": "Apples, bananas, oranges, strawberries, lemon juice, sugar",
        "instructions": "Cut the fruit into pieces, mix with lemon juice and sugar, refrigerate.",
        "expected_vegan": True
    },
    {
        "ingredients": "Chicken breast, onion, curry powder, cream or coconut milk, oil, salt",
        "instructions": "Brown the onion, add diced chicken, curry, and cream. Cook until the chicken is tender.",
        "expected_vegan": False
    },
    {
        "ingredients": "Ladyfingers, mascarpone, eggs, sugar, coffee, cocoa powder",
        "instructions": "Make a cream with yolks, sugar and mascarpone. Layer ladyfingers soaked in coffee and the cream. Dust with cocoa and chill.",
        "expected_vegan": False
    },
    {
        "ingredients": "2 eggs, 250ml milk, 125g flour, sugar, butter",
        "instructions": "Mix flour, eggs and milk into a batter. Cook in pan with butter. Fill as desired.",
        "expected_vegan": False
    },
    {
        "ingredients": "Mixed dried legumes, onion, carrot, celery, vegetable broth, oil, salt",
        "instructions": "Soak legumes for 12 hours. Sauté onion, carrot, celery. Add legumes and broth and cook for at least an hour.",
        "expected_vegan": True
    }
]

# Prepare input texts for models (ingredients + instructions)
texts = [r["ingredients"] + " " + r["instructions"] for r in recipes]

# Tokenize and padding
vegan_sequences = vegan_tokenizer.texts_to_sequences(texts)
vegan_padded = pad_sequences(vegan_sequences, maxlen=MAX_LEN, padding='post', truncating='post')

# Predicting vegan true or false
vegan_preds = vegan_model.predict(vegan_padded)
vegan_classes = vegan_preds.argmax(axis=1)
vegan_labels = [inv_vegan_label_map[c] for c in vegan_classes]

# Print results
for i, recipe in enumerate(recipes):
    print(f"Recipe {i+1}:")
    print(f"  Expected vegan: {recipe['expected_vegan']}")
    print(f"  Predicted vegan: {vegan_labels[i]}")
    vegan_probs_str = ", ".join([f"{inv_vegan_label_map[j]}: {vegan_preds[i][j]*100:.2f}%" for j in range(len(vegan_preds[i]))])
    print(f"  Vegan probabilities: {vegan_probs_str}\n")

# Creazione del modello per la classificazione del tag 'VEGETARIAN' e testing

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import pickle

# Default parameters
MAX_LEN = 500
NUM_WORDS = 20000
BATCH_SIZE = 32
EPOCHS = 10

def preprocess(df, text_columns, label_column, n=10000):
    df = df[text_columns + [label_column]].dropna().head(n).copy()
    df[text_columns[0]] = df[text_columns[0]].apply(eval)
    df[text_columns[1]] = df[text_columns[1]].apply(eval)
    df["full_text"] = df[text_columns[0]].apply(lambda x: " ".join(x)) + " " + df[text_columns[1]].apply(lambda x: " ".join(x))

    label_encoder = LabelEncoder()
    df["encoded_label"] = label_encoder.fit_transform(df[label_column])

    return df, label_encoder

def train_text_classifier(df, label_encoder, model_name_prefix):
    X = df["full_text"]
    y = df["encoded_label"]

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

    tokenizer = Tokenizer(num_words=NUM_WORDS, oov_token="<OOV>")
    tokenizer.fit_on_texts(X_train)

    X_train_seq = tokenizer.texts_to_sequences(X_train)
    X_val_seq = tokenizer.texts_to_sequences(X_val)

    X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding="post", truncating="post")
    X_val_pad = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding="post", truncating="post")

    model = Sequential([
        Embedding(input_dim=NUM_WORDS, output_dim=128, input_length=MAX_LEN),
        GlobalMaxPooling1D(),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dense(len(label_encoder.classes_), activation='softmax')
    ])

    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.fit(
        X_train_pad, y_train,
        validation_data=(X_val_pad, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=[early_stop]
    )

    model.save(f"classifiers/vegetarian/{model_name_prefix}.h5")
    with open(f"classifiers/vegetarian/{model_name_prefix}_label_mapping.pkl", "wb") as f:
        pickle.dump(dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))), f)

    with open(f"classifiers/vegetarian/{model_name_prefix}_tokenizer.pkl", "wb") as f:
        pickle.dump(tokenizer, f)

    print(f"Modello '{model_name_prefix}' salvato con successo.")
    return model

# === FOURTH MODEL: VEGETARIAN ===
df_vegetarian = df[["INGREDIENTS", "DIRECTIONS", "VEGETARIAN"]].copy()
df_vegetarian, vegetarian_encoder = preprocess(df_vegetarian, ["INGREDIENTS", "DIRECTIONS"], "VEGETARIAN")
vegetarian_classifier_model = train_text_classifier(df_vegetarian, vegetarian_encoder, model_name_prefix="vegetarian_classifier")

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

MAX_LEN = 500  # same as training

# Load model and tokenizer
vegetarian_model = load_model("classifiers/vegetarian/vegetarian_classifier.h5")

with open("classifiers/vegetarian/vegetarian_classifier_tokenizer.pkl", "rb") as f:
    vegetarian_tokenizer = pickle.load(f)

with open("classifiers/vegetarian/vegetarian_classifier_label_mapping.pkl", "rb") as f:
    vegetarian_label_map = pickle.load(f)

# Reverse dictionaries to decode predictions
inv_vegetarian_label_map = {v: k for k, v in vegetarian_label_map.items()}

# List of 10 recipes with expected vegetarian
recipes = [
    {
        "ingredients": "200g spaghetti, 2 cloves garlic, chili pepper, extra virgin olive oil, salt",
        "instructions": "Cook the spaghetti. Meanwhile, sauté garlic and chili in the oil. Mix with the pasta.",
        "expected_vegetarian": True
    },
    {
        "ingredients": "Lettuce, 1 can of tuna, tomatoes, black olives, onion, olive oil, salt",
        "instructions": "Wash and cut the vegetables. Add the drained tuna and olives. Dress with oil and salt.",
        "expected_vegetarian": False  # Tuna = non vegetarian
    },
    {
        "ingredients": "300g Carnaroli rice, 1 saffron sachet, vegetable broth, butter, onion, Parmesan cheese",
        "instructions": "Sauté the onion, add rice and deglaze with wine. Gradually pour in the broth. Add saffron and stir in butter and cheese.",
        "expected_vegetarian": True
    },
    {
        "ingredients": "3 eggs, salt, pepper, butter",
        "instructions": "Beat the eggs with salt and pepper. Pour into a pan with melted butter and stir until set.",
        "expected_vegetarian": True
    },
    {
        "ingredients": "Lasagna sheets, meat sauce, béchamel, Parmesan cheese",
        "instructions": "Layer pasta, sauce, béchamel and cheese. Bake at 180°C for 40 minutes.",
        "expected_vegetarian": False  # Meat sauce = non vegetarian
    },
    {
        "ingredients": "Apples, bananas, oranges, strawberries, lemon juice, sugar",
        "instructions": "Cut the fruit into pieces, mix with lemon juice and sugar, refrigerate.",
        "expected_vegetarian": True
    },
    {
        "ingredients": "Chicken breast, onion, curry powder, cream or coconut milk, oil, salt",
        "instructions": "Brown the onion, add diced chicken, curry, and cream. Cook until the chicken is tender.",
        "expected_vegetarian": False  # Chicken = non vegetarian
    },
    {
        "ingredients": "Ladyfingers, mascarpone, eggs, sugar, coffee, cocoa powder",
        "instructions": "Make a cream with yolks, sugar and mascarpone. Layer ladyfingers soaked in coffee and the cream. Dust with cocoa and chill.",
        "expected_vegetarian": True
    },
    {
        "ingredients": "2 eggs, 250ml milk, 125g flour, sugar, butter",
        "instructions": "Mix flour, eggs and milk into a batter. Cook in pan with butter. Fill as desired.",
        "expected_vegetarian": True
    },
    {
        "ingredients": "Mixed dried legumes, onion, carrot, celery, vegetable broth, oil, salt",
        "instructions": "Soak legumes for 12 hours. Sauté onion, carrot, celery. Add legumes and broth and cook for at least an hour.",
        "expected_vegetarian": True
    }
]

# Prepare input texts for models (ingredients + instructions)
texts = [r["ingredients"] + " " + r["instructions"] for r in recipes]

# Tokenize and padding
vegetarian_sequences = vegetarian_tokenizer.texts_to_sequences(texts)
vegetarian_padded = pad_sequences(vegetarian_sequences, maxlen=MAX_LEN, padding='post', truncating='post')

# Predicting vegetarian true or false
vegetarian_preds = vegetarian_model.predict(vegetarian_padded)
vegetarian_classes = vegetarian_preds.argmax(axis=1)
vegetarian_labels = [inv_vegetarian_label_map[c] for c in vegetarian_classes]

# Print results
for i, recipe in enumerate(recipes):
    print(f"Recipe {i+1}:")
    print(f"  Expected vegetarian: {recipe['expected_vegetarian']}")
    print(f"  Predicted vegetarian: {vegetarian_labels[i]}")
    vegetarian_probs_str = ", ".join([f"{inv_vegetarian_label_map[j]}: {vegetarian_preds[i][j]*100:.2f}%" for j in range(len(vegetarian_preds[i]))])
    print(f"  Vegetarian probabilities: {vegetarian_probs_str}\n")

# Creazione del modello per la classificazione del tag 'METHOD' e testing

In [17]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import pickle

# Default parameters
MAX_LEN = 500
NUM_WORDS = 20000
BATCH_SIZE = 32
EPOCHS = 10

def preprocess(df, text_columns, label_column, n=10000):
    df = df[text_columns + [label_column]].dropna().head(n).copy()
    df[text_columns[0]] = df[text_columns[0]].apply(eval)
    df[text_columns[1]] = df[text_columns[1]].apply(eval)
    df["full_text"] = df[text_columns[0]].apply(lambda x: " ".join(x)) + " " + df[text_columns[1]].apply(lambda x: " ".join(x))

    label_encoder = LabelEncoder()
    df["encoded_label"] = label_encoder.fit_transform(df[label_column])

    return df, label_encoder

def train_text_classifier(df, label_encoder, model_name_prefix):
    X = df["full_text"]
    y = df["encoded_label"]

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

    tokenizer = Tokenizer(num_words=NUM_WORDS, oov_token="<OOV>")
    tokenizer.fit_on_texts(X_train)

    X_train_seq = tokenizer.texts_to_sequences(X_train)
    X_val_seq = tokenizer.texts_to_sequences(X_val)

    X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding="post", truncating="post")
    X_val_pad = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding="post", truncating="post")

    model = Sequential([
        Embedding(input_dim=NUM_WORDS, output_dim=128, input_length=MAX_LEN),
        GlobalMaxPooling1D(),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dense(len(label_encoder.classes_), activation='softmax')
    ])

    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.fit(
        X_train_pad, y_train,
        validation_data=(X_val_pad, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=[early_stop]
    )

    model.save(f"classifiers/method/{model_name_prefix}.h5")
    with open(f"classifiers/method/{model_name_prefix}_label_mapping.pkl", "wb") as f:
        pickle.dump(dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))), f)

    with open(f"classifiers/method/{model_name_prefix}_tokenizer.pkl", "wb") as f:
        pickle.dump(tokenizer, f)

    print(f"Modello '{model_name_prefix}' salvato con successo.")
    return model

# === FIFTH MODEL: METHOD ===
df_method = df[["INGREDIENTS", "DIRECTIONS", "method"]].copy()
df_method, method_encoder = preprocess(df_method, ["INGREDIENTS", "DIRECTIONS"], "method")
method_classifier_model = train_text_classifier(df_method, method_encoder, model_name_prefix="method_classifier")

Epoch 1/10


C:\Users\aless\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


250/250 ━━━━━━━━━━━━━━━━━━━━ 10s 36ms/step - accuracy: 0.4803 - loss: 1.5450 - val_accuracy: 0.8685 - val_loss: 0.6252
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - accuracy: 0.8676 - loss: 0.5842 - val_accuracy: 0.9145 - val_loss: 0.4036
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 35ms/step - accuracy: 0.9044 - loss: 0.4103 - val_accuracy: 0.9195 - val_loss: 0.3357
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 34ms/step - accuracy: 0.9136 - loss: 0.3366 - val_accuracy: 0.9215 - val_loss: 0.2983
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 33ms/step - accuracy: 0.9233 - loss: 0.2860 - val_accuracy: 0.9225 - val_loss: 0.2683
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9296 - loss: 0.2529 - val_accuracy: 0.9300 - val_loss: 0.2578
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9324 - loss: 0.2210 - val_accuracy: 0.9320 - val_loss: 0.2335
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9376 - loss: 0.2054 - val_accuracy: 0

Modello 'method_classifier' salvato con successo.


In [18]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

MAX_LEN = 500  # same as training

# Load model and tokenizer
method_model = load_model("classifiers/method/method_classifier.h5")

with open("classifiers/method/method_classifier_tokenizer.pkl", "rb") as f:
    method_tokenizer = pickle.load(f)

with open("classifiers/method/method_classifier_label_mapping.pkl", "rb") as f:
    method_label_map = pickle.load(f)

# Reverse dictionaries to decode predictions
inv_method_label_map = {v: k for k, v in method_label_map.items()}

# List of 10 recipes with expected method
recipes = [
    {
        "ingredients": "500g potatoes, olive oil, rosemary, salt",
        "instructions": "Peel and cut the potatoes into chunks. Toss with olive oil and rosemary. Cook in oven at 200°C for 45 minutes.",
        "expected_method": "Roasted"
    },
    {
        "ingredients": "200g spaghetti, salt, water",
        "instructions": "Boil water in a large pot, add salt and cook the spaghetti until al dente.",
        "expected_method": "Boiled"
    },
    {
        "ingredients": "2 chicken breasts, olive oil, herbs, salt",
        "instructions": "Brush chicken with oil and herbs. Cook on a preheated grill for 6 minutes per side.",
        "expected_method": "Grilled"
    },
    {
        "ingredients": "4 eggs, butter, salt, pepper",
        "instructions": "Heat butter in a pan. Crack eggs in and fry until edges are crispy.",
        "expected_method": "Fried"
    },
    {
        "ingredients": "1 salmon fillet, lemon juice, herbs, salt",
        "instructions": "Place salmon in a baking dish, season, and bake at 180°C for 20 minutes.",
        "expected_method": "Baked"
    },
    {
        "ingredients": "Broccoli, carrots, zucchini, salt",
        "instructions": "Wash and cut vegetables. Eat raw or season with salt and olive oil.",
        "expected_method": "Raw"
    },
    {
        "ingredients": "Beef steak, salt, pepper, olive oil",
        "instructions": "Season steak and place under a broiler for 5–7 minutes on each side.",
        "expected_method": "Broil"
    },
    {
        "ingredients": "1 cauliflower, salt, oil, paprika",
        "instructions": "Cut cauliflower into florets. Toss with oil and paprika. Roast in oven at 220°C for 30 minutes.",
        "expected_method": "Roasted"
    },
    {
        "ingredients": "Rice, water, salt",
        "instructions": "Rinse the rice and boil in salted water for 15 minutes.",
        "expected_method": "Boil"
    },
    {
        "ingredients": "1 can chickpeas, tahini, lemon juice, garlic, olive oil",
        "instructions": "Blend all ingredients in a food processor until smooth. Serve as is.",
        "expected_method": "Other"  # Nessuna cottura classica, solo preparazione a crudo e frullatura
    }
]

# Prepare input texts for models (ingredients + instructions)
texts = [r["ingredients"] + " " + r["instructions"] for r in recipes]

# Tokenize and padding
method_sequences = method_tokenizer.texts_to_sequences(texts)
method_padded = pad_sequences(method_sequences, maxlen=MAX_LEN, padding='post', truncating='post')

# Predicting method
method_preds = method_model.predict(method_padded)
method_classes = method_preds.argmax(axis=1)
method_labels = [inv_method_label_map[c] for c in method_classes]

# Print results
for i, recipe in enumerate(recipes):
    print(f"Recipe {i+1}:")
    print(f"  Expected method: {recipe['expected_method']}")
    print(f"  Predicted method: {method_labels[i]}")
    method_probs_str = ", ".join([f"{inv_method_label_map[j]}: {method_preds[i][j]*100:.2f}%" for j in range(len(method_preds[i]))])
    print(f"  Method probabilities: {method_probs_str}\n")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
Recipe 1:
  Expected method: Roasted
  Predicted method: Other
  Method probabilities: Bake: 0.62%, Baked: 0.05%, Boil: 1.92%, Boiled: 0.73%, Broil: 2.13%, Fried: 0.75%, Grill: 0.63%, Grilled: 0.07%, Other: 90.87%, Raw: 0.40%, Roasted: 1.84%

Recipe 2:
  Expected method: Boiled
  Predicted method: Boil
  Method probabilities: Bake: 0.27%, Baked: 0.12%, Boil: 96.48%, Boiled: 1.41%, Broil: 0.01%, Fried: 0.54%, Grill: 0.00%, Grilled: 0.05%, Other: 0.03%, Raw: 0.56%, Roasted: 0.53%

Recipe 3:
  Expected method: Grilled
  Predicted method: Grill
  Method probabilities: Bake: 0.01%, Baked: 0.02%, Boil: 0.19%, Boiled: 0.08%, Broil: 0.19%, Fried: 0.11%, Grill: 98.71%, Grilled: 0.61%, Other: 0.02%, Raw: 0.04%, Roasted: 0.02%

Recipe 4:
  Expected method: Fried
  Predicted method: Other
  Method probabilities: Bake: 0.02%, Baked: 0.00%, Boil: 0.22%, Boiled: 0.04%, Broil: 0.13%, Fried: 0.04%, Grill: 0.09%, Grilled: 0.00%, Other: 99.34%, Raw: 0.05%, Roasted: 0